# WhisperX48

<font size="3">将 WhisperX 部署在 Google Colab 云端上，其目标是减少视频字幕制作过程中听译和打轴的繁重工作。详细信息和帮助文档可查阅 [README](https://github.com/ifeimi/Whisper48/blob/main/README.md) 文件和[我的主页](https://ifeimi.github.io/whisper48/)。  
This IPython Notebook is designed as an implementation of WhisperX on Google Colab. The application serves to reduce the heavy and tedious work in transcription and timestamping in video-subtitling. Detailed information and help document can be found in [README](https://github.com/ifeimi/Whisper48/blob/main/README.md) and on [my website](https://ifeimi.github.io/whisper48/).   
\
如果本文档不能正常运行，可以尝试基于 Whisper的替代品 [Whisper48](https://github.com/ifeimi/Whisper48/blob/main/Whisper48.ipynb) 或 [N46Whisper](https://github.com/Ayanaminn/N46Whisper)。
If this notebook does not work properly, you can try the alternatives based on Whisper: [Whisper48]((https://github.com/ifeimi/Whisper48/blob/main/Whisper48.ipynb) or [N46Whisper](https://github.com/Ayanaminn/N46Whisper).
\
请按提示依次执行以下单元格，建议在开始前先将需要转录的音频文件上传到谷歌网盘中。  
Please run the following cells in order according to the help text. It is suggested to upload your audio file to Google Drive first before you start.  
\
联系作者/Contact me: ifeimi48@gmail.com.<font size="3">  

In [ ]:
#@markdown **1.1 挂载谷歌云盘 / Mount Google Drive (approx. 0.5 min)**
#@markdown **</br><font size="2">【重要】确保在"运行时"→"更改运行时类型"中选择了GPU作为硬件加速器 / 【IMPORTANT】 Make sure you have selected GPU as the hardware accelerator in "Runtime" → "Change runtime type"</font><br/>

from google.colab import drive, files
from IPython.display import clear_output
import os

# 挂载谷歌云盘以访问媒体文件 / Mount Google Drive to access media files
clear_output()
print("=" * 60)
print("步骤 1.1: 挂载谷歌云盘 / STEP 1.1: Mounting Google Drive")
print("=" * 60)
print("请在弹出窗口中授权连接 / Please authorize the connection in the pop-up window...")
drive.mount('/drive')
clear_output()
print("✓ 谷歌云盘挂载成功！/ Google Drive mounted successfully!")
print("准备好时请执行下一步 / Proceed to the next step when ready.")


In [3]:
#@markdown **1.2 配置运行环境 / Setup Environment (approx. 3 min)**

import subprocess
from IPython.display import clear_output

print("=" * 60)
print("步骤 1.2: 配置环境并安装依赖 / STEP 1.2: Setting up environment and installing dependencies")
print("=" * 60)

# 修复 CUDA 兼容性的 CuDNN 库路径 / Fix CuDNN library path for CUDA compatibility
print("\n[1/3] 修复 CuDNN 库配置 / Fixing CuDNN library configuration...")
subprocess.run('ln -s /usr/local/lib/python3.12/dist-packages/nvidia/cudnn/lib/libcudnn* "$LD_LIBRARY_PATH"/ 2>/dev/null || true', shell=True)

# 安装所需的包 / Install required packages
print("[2/3] 安装 ipytree 文件浏览器 / Installing ipytree for file browser...")
subprocess.run('pip install ipytree -q', shell=True)

print("[3/3] 安装 whisperX")
subprocess.run('pip uninstall -y numpy pandas whisperx numba tensorflow cudf dask-cudf gradio fastai',shell=True)
subprocess.run('pip install whisperx',shell=True)


clear_output()
print("✓ 环境配置完成！/ Environment setup completed successfully!")
print("准备好时请执行下一步 / Proceed to the next step when ready.")

✓ 环境配置完成！/ Environment setup completed successfully!
准备好时请执行下一步 / Proceed to the next step when ready.


In [2]:
#@markdown **1.3 从谷歌云盘选择文件 / Select File from Google Drive (0 min)**
# @markdown <br/><font size="2"><b>使用说明 / Instructions:</b>
# @markdown <br/>1. 在目录树中导航到您的媒体文件（音频/视频）/ Navigate to your media file (audio/video) in the directory tree
# @markdown <br/>2. 点击选择文件 / Click to select the file
# @markdown <br/>3. 点击"Select"按钮确认选择 / Click the 'Select' button to confirm your choice
# @markdown <br/><br/><b>注意 / Note:</b> 如果您在此时才上传了文件，请再次运行此单元格以刷新文件列表。 / If you just uploaded a file, run this cell again to refresh the file list.
# @markdown </font>

from ipytree import Tree, Node
import ipywidgets as widgets
from google.colab import output
from IPython.display import clear_output

output.enable_custom_widget_manager()

# 全局变量用于存储选定的文件路径 / Global variable to store selected file path
selected_file_path = ''

def create_file_browser():
    """创建交互式文件浏览器小部件，用于从谷歌云盘选择媒体文件。
    Create an interactive file browser widget for selecting media files from Google Drive."""

    # 初始化小部件 / Initialize widgets
    full_widget = widgets.HBox()
    left_widget = widgets.VBox()
    right_widget = widgets.VBox()

    path_widget = widgets.Text(
        description='已选择 / Selected:',
        layout=widgets.Layout(min_width='300px')
    )

    select_button = widgets.Button(
        description='Select',
        button_style='success',
        tooltip='确认并选择当前媒体文件 / Confirm and select the current media file'
    )

    # 支持的媒体格式 / Supported media formats
    SUPPORTED_FORMATS = ('mp3', 'm4a', 'flac', 'aac', 'wav', 'mp4', 'mkv', 'ts', 'flv')

    # 创建文件树小部件 / Create file tree widget
    tree_output = widgets.Output(layout=widgets.Layout(max_width='300px', max_height='400px', overflow='auto'))
    tree_widget = Tree(multiple_selection=False)
    tree_dict = {}

    left_widget.children = [
        widgets.Label('浏览到您的媒体文件 / Browse to your media file:'),
        path_widget,
        tree_output
    ]
    right_widget.children = [select_button]
    full_widget.children = [left_widget]

    def on_select_click(button):
        """处理选择确认 / Handle selection confirmation."""
        global selected_file_path
        selected_file_path = path_widget.value
        clear_output()
        print("=" * 60)
        print("✓ 文件选择成功！/ File selected successfully!")
        print("=" * 60)
        print(f"已选择文件 / Selected file: {selected_file_path}")
        print("\n准备好时请执行下一步 / Proceed to the next step.")

    def on_file_click(event):
        """处理树中的文件选择 / Handle file selection in the tree."""
        if event['new']:
            current_node = event['owner']
            for file_path, node in tree_dict.items():
                if current_node is node and os.path.isfile(file_path):
                    path_widget.value = file_path
                    path_widget.disabled = False
                    select_button.disabled = False
                    full_widget.children = [left_widget, right_widget]
                    return

    def on_folder_click(event):
        """处理树中的文件夹选择 / Handle folder selection in the tree."""
        if event['new']:
            full_widget.children = [left_widget]

    select_button.on_click(on_select_click)

    # 构建文件树 / Build file tree
    def build_tree(root_path):
        """递归构建媒体文件的树形结构 / Recursively build tree structure of media files."""
        root_name = os.path.basename(root_path) or 'MyDrive'
        root_node = Node(root_name)
        tree_dict[root_path] = root_node
        tree_widget.add_node(root_node)
        root_node.observe(on_folder_click, 'selected')

        # 遍历目录 / Walk through directory
        for current_dir, subdirs, files_list in os.walk(root_path):
            # 过滤隐藏文件夹 / Filter hidden folders
            subdirs[:] = [d for d in subdirs if not d.startswith('.')]

            # 为子目录创建节点 / Create nodes for subdirectories
            for subdir in sorted(subdirs):
                subdir_path = os.path.join(current_dir, subdir)
                if subdir_path not in tree_dict:
                    parent_node = tree_dict[current_dir]
                    subdir_node = Node(subdir)
                    tree_dict[subdir_path] = subdir_node
                    parent_node.add_node(subdir_node)
                    subdir_node.observe(on_folder_click, 'selected')

            # 为媒体文件创建节点 / Create nodes for media files
            media_files = [f for f in sorted(files_list) if f.lower().endswith(SUPPORTED_FORMATS)]
            if media_files and current_dir in tree_dict:
                parent_node = tree_dict[current_dir]
                parent_node.opened = False
                for media_file in media_files:
                    file_path = os.path.join(current_dir, media_file)
                    file_node = Node(media_file)
                    file_node.icon = 'file'
                    tree_dict[file_path] = file_node
                    parent_node.add_node(file_node)
                    file_node.observe(on_file_click, 'selected')

    # 使用谷歌云盘初始化树 / Initialize tree with Google Drive
    drive_root = '/drive/MyDrive'
    if os.path.exists(drive_root):
        build_tree(drive_root)

    # 显示树 / Display tree
    with tree_output:
        tree_output.clear_output()
        display(tree_widget)

    return full_widget

# 创建并显示文件浏览器 / Create and display the file browser
print("=" * 60)
print("步骤 1.3: 选择媒体文件 / STEP 1.3: Select Media File")
print("=" * 60)
browser = create_file_browser()
display(browser)


✓ 文件选择成功！/ File selected successfully!
已选择文件 / Selected file: /drive/MyDrive/花薰-02-1080p_extracted.wav

准备好时请执行下一步 / Proceed to the next step.


In [1]:
#@markdown **2.1 配置参数 / Configure Parameters (0 min)**
# @markdown <br/><font size="3">**2.1.1 输入文件类型 / Input File Type**</font>
# @markdown <br/><font size="2">选择您的文件是视频还是音频文件 / Select whether your file is a video or audio file</font>

file_type = "audio"  # @param ["audio","video"]

# @markdown <br/><br/><font size="3">**2.1.2 模型选择 / Model Selection**</font>
# @markdown <br/><font size="2">选择 Whisper 模型大小和目标语言 / Choose the Whisper model size and target language:
# @markdown <br/>- 较大的模型精度更高但速度较慢且消耗更多 GPU 内存 / Larger models are more accurate but slower and consume more GPU memory
# @markdown <br/>- 语言检测有助于模型优化 / Language detection helps with model optimization</font>

model_size = "large-v3"  # @param ["base","small","medium", "large-v1","large-v2","large-v3"]
language = "ja"  # @param ["ja","zh","en","fr", "de","es","it","pt","ru"]

# @markdown <br/><br/><font size="3">**2.1.3 高级设置 / Advanced Settings**</font>
# @markdown <br/><font size="2"><b>如果您不确定这些设置的含义，请保持默认值。/ Only modify these if you know what you're doing. Otherwise, keep defaults.</b></font>

max_line_width = "None"  # @param {type:"string"} - 每行字幕的最大字符数（None 表示自动）/ Maximum characters per subtitle line (None for auto)
max_line_count = "None"  # @param {type:"string"} - 每个字幕块的最大行数（None 表示自动）/ Maximum number of lines per subtitle block (None for auto)
highlight_words = False  # @param ["False","True"] - 为每个单词分别突出显示和计时 / Highlight each word with separate timing
chunk_size = 5  # @param {type:"integer"} - 音频处理的块大小（秒）/ Audio chunk size for processing (seconds)
TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD = "1"  # @param ["1","0"] - PyTorch 权重加载模式 / PyTorch weight loading mode

print("=" * 60)
print("步骤 2.1: 配置 / STEP 2.1: Configuration")
print("=" * 60)
print(f"\n文件类型 / File Type: {file_type}")
print(f"模型大小 / Model Size: {model_size}")
print(f"语言 / Language: {language}")
print(f"最大行宽 / Max Line Width: {max_line_width}")
print(f"最大行数 / Max Line Count: {max_line_count}")
print(f"块大小 / Chunk Size: {chunk_size}s")

# 设置环境变量 / Set environment variables
import os
os.environ["TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD"] = TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD

# 解析并验证 max_line_width
if max_line_width is not None:
    try:
        max_line_width = int(max_line_width)
    except ValueError:
        if max_line_width.lower() == "none":
            max_line_width = None
        else:
            print("⚠ 警告 / Warning: max_line_width 无效。使用 None（自动）/ is invalid. Using None (auto).")
            max_line_width = None

# 解析并验证 max_line_count
if max_line_count is not None:
    try:
        max_line_count = int(max_line_count)
    except ValueError:
        if max_line_count.lower() == "none":
            max_line_count = None
        else:
            print("⚠ 警告 / Warning: max_line_count 无效。使用 None（自动）/ is invalid. Using None (auto).")
            max_line_count = None

print("\n✓ 配置验证成功！/ Configuration validated successfully!")


步骤 2.1: 配置 / STEP 2.1: Configuration

文件类型 / File Type: audio
模型大小 / Model Size: large-v3
语言 / Language: ja
最大行宽 / Max Line Width: None
最大行数 / Max Line Count: None
块大小 / Chunk Size: 5s

✓ 配置验证成功！/ Configuration validated successfully!


In [3]:
#@markdown **2.2 运行 WhisperX 转录 / Run WhisperX Transcription (approx. 5-20 min)**
#@markdown <br/><font size="2">该过程包括：/ The process includes:
#@markdown <br/>1. 音频提取（如提供视频文件）/ Audio extraction (if video file provided)
#@markdown <br/>2. 使用 Whisper 模型进行语音转文字 / Speech-to-text transcription using Whisper model
#@markdown <br/>3. 使用 wav2vec2.0 进行单词级时间戳对齐 / Word-level timestamp alignment using wav2vec2.0
#@markdown <br/><br/>SRT 字幕文件完成后将自动下载到您的计算机。/ The SRT subtitle file will be automatically downloaded to your computer.</font>

import torch
import whisperx
import time
import gc
import os
from whisperx.utils import WriteSRT
from google.colab import files

print("=" * 60)
print("步骤 2.2: 运行 WhisperX 转录 / STEP 2.2: Running WhisperX Transcription")
print("=" * 60)

# 验证是否选择了文件 / Validate that a file was selected
if not selected_file_path or not os.path.exists(selected_file_path):
    print("❌ 错误 / ERROR: 未选择有效文件。请返回步骤 1.3 并选择文件。/ No valid file selected. Please go back to step 1.3 and select a file.")
else:
    try:
        # 设置文件路径 / Setup file paths
        file_name = selected_file_path
        file_basename = os.path.splitext(file_name)[0]
        output_dir = os.path.dirname(file_name)

        print(f"\n输入文件 / Input file: {file_name}")
        print(f"输出目录 / Output directory: {output_dir}\n")

        # ========== 步骤 1: 如需从视频提取音频 / STEP 1: Extract audio from video if needed ==========
        if file_type == "video":
            print("[1/4] 从视频文件提取音频 / Extracting audio from video file...")
            audio_file = f'{file_basename}_extracted.wav'
            os.system(f'ffmpeg -i "{file_name}" -ar 16000 -ac 1 -c:a pcm_s16le "{audio_file}" 2>/dev/null')
            print(f"✓ 音频已提取 / Audio extracted: {audio_file}\n")
        else:
            audio_file = file_name
            print("[1/4] 直接使用音频文件 / Using audio file directly...\n")

        # ========== 步骤 2: 加载模型并转录 / STEP 2: Load model and transcribe ==========
        device = "cuda"
        batch_size = 16  # 如果 GPU 内存不足，请减少此值 / Reduce if GPU runs out of memory
        compute_type = "float16"  # 如果 GPU 内存不足，使用 "int8"（可能会降低精度）/ Use "int8" if low on GPU memory (may reduce accuracy)

        print("[2/4] 加载 Whisper 模型（这可能需要一段时间）/ Loading Whisper model (this may take a moment)...")
        model = whisperx.load_model(model_size, device, compute_type=compute_type, language=language)
        print(f"✓ 模型已加载 / Model loaded: {model_size}\n")

        print("[2.5/4] 加载音频文件 / Loading audio file...")
        audio = whisperx.load_audio(audio_file)
        print(f"✓ 音频加载成功 / Audio loaded successfully\n")

        print("[3/4] 运行语音识别转录（可能需要几分钟）/ Running speech-to-text transcription (may take several minutes)...")
        start_time = time.time()
        result = model.transcribe(audio, batch_size=batch_size, chunk_size=chunk_size)
        transcription_time = time.time() - start_time
        print(f"✓ 转录完成，耗时 / Transcription completed in {transcription_time:.1f}s\n")

        # 保存初始转录（对齐前）/ Save initial transcription (before alignment)
        transcription_srt = f'{file_basename}_transcription.srt'
        options = {
            "max_line_width": max_line_width,
            "max_line_count": max_line_count,
            "highlight_words": highlight_words
        }
        with open(transcription_srt, "w", encoding="utf-8") as f:
            srt_writer = WriteSRT(transcription_srt)
            srt_writer.write_result(result, f, options)
        print(f"✓ 初始转录已保存 / Initial transcription saved: {transcription_srt}\n")

        # 清理以释放 GPU 内存 / Clean up to free GPU memory
        gc.collect()
        torch.cuda.empty_cache()
        del model

        # ========== 步骤 3: 对齐时间戳以获得单词级精度 / STEP 3: Align timestamps for word-level accuracy ==========
        print("[3.5/4] 加载对齐模型（wav2vec2.0）/ Loading alignment model (wav2vec2.0)...")
        alignment_model, metadata = whisperx.load_align_model(language_code=language, device=device)
        print(f"✓ 对齐模型已加载 / Alignment model loaded\n")

        print("[4/4] 执行单词级时间戳对齐 / Performing word-level timestamp alignment...")
        result_aligned = whisperx.align(
            result["segments"],
            alignment_model,
            metadata,
            audio,
            device,
            return_char_alignments=False
        )
        result_aligned["language"] = result["language"]
        print(f"✓ 对齐完成 / Alignment completed\n")

        # 清理 / Clean up
        gc.collect()
        torch.cuda.empty_cache()
        del alignment_model

        # ========== 步骤 4: 保存最终 SRT 文件 / STEP 4: Save final SRT file ==========
        final_srt = f'{file_basename}.srt'
        with open(final_srt, "w", encoding="utf-8") as f:
            srt_writer = WriteSRT(final_srt)
            srt_writer.write_result(result_aligned, f, options)

        print(f"[完成 / COMPLETE] 最终字幕文件 / Final subtitle file: {final_srt}")
        print("\n" + "=" * 60)
        print("✓ 转录和对齐成功！/ TRANSCRIPTION AND ALIGNMENT SUCCESSFUL!")
        print("=" * 60)
        print(f"\n总处理时间 / Total processing time: {transcription_time:.1f}s")
        print(f"输出文件 / Output file: {final_srt}\n")

        # 下载文件 / Download the file
        print("正在下载字幕文件 / Downloading subtitle file...")
        files.download(final_srt)
        print("✓ 下载已开始！请检查您的下载文件夹。/ Download started! Check your Downloads folder.")

    except Exception as e:
        print(f"\n❌ 错误 / ERROR: {str(e)}")
        print("\n请检查以下内容 / Please check:")
        print("1. 选定的文件路径是否正确？/ Is the selected file path correct?")
        print("2. 您是否有足够的 GPU 内存？/ Do you have sufficient GPU memory?")
        print("3. 音频格式是否受支持？/ Is the audio format supported?")


步骤 2.2: 运行 WhisperX 转录 / STEP 2.2: Running WhisperX Transcription

输入文件 / Input file: /drive/MyDrive/花薰-02-1080p_extracted.wav
输出目录 / Output directory: /drive/MyDrive

[1/4] 直接使用音频文件 / Using audio file directly...

[2/4] 加载 Whisper 模型（这可能需要一段时间）/ Loading Whisper model (this may take a moment)...


/usr/local/lib/python3.12/dist-packages/pyannote/audio/core/io.py:212: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  torchaudio.list_audio_backends()
/usr/local/lib/python3.12/dist-packages/speechbrain/utils/torch_audio_backend.py:57: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed fr

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocabulary.json: 0.00B [00:00, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

✓ 对齐模型已加载 / Alignment model loaded

[4/4] 执行单词级时间戳对齐 / Performing word-level timestamp alignment...
2026-02-05 13:53:44 - whisperx.alignment - WARNING - Failed to align segment ("ご視聴ありがとうございました"): backtrack failed, resorting to original
✓ 对齐完成 / Alignment completed

[完成 / COMPLETE] 最终字幕文件 / Final subtitle file: /drive/MyDrive/花薰-02-1080p_extracted.srt

✓ 转录和对齐成功！/ TRANSCRIPTION AND ALIGNMENT SUCCESSFUL!

总处理时间 / Total processing time: 113.9s
输出文件 / Output file: /drive/MyDrive/花薰-02-1080p_extracted.srt

正在下载字幕文件 / Downloading subtitle file...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✓ 下载已开始！请检查您的下载文件夹。/ Download started! Check your Downloads folder.
